In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import shap
from linearmodels.panel import PanelOLS, RandomEffects
from sklearn.inspection import PartialDependenceDisplay
import os
import warnings
from scipy import stats
from statsmodels.regression.quantile_regression import QuantReg
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from statsmodels.iolib.summary2 import summary_col

# File path
file_path = r"C:\Users\jiaju\Desktop\Final_data.xlsx"

# Load Sheet1
data = pd.read_excel(file_path, sheet_name="Sheet1")

In [33]:
# drop rows with the same Date & Company, keep the first
df = data.drop_duplicates(subset=['Date', 'COMPANY'], keep='first')

In [4]:
# 将 Date 列转换为日期格式
def parse_quarter(date_str):
    quarter, year = date_str.split()
    year = int(year)
    if quarter == 'Q1':
        return f'{year}-01-01'
    elif quarter == 'Q2':
        return f'{year}-04-01'
    elif quarter == 'Q3':
        return f'{year}-07-01'
    elif quarter == 'Q4':
        return f'{year}-10-01'
    return None

data['Date'] = pd.to_datetime(data['Date'].apply(parse_quarter), errors='coerce')
print("Date 列类型：", data['Date'].dtype)
print("Date 列前五行：\n", data['Date'].head())

Date 列类型： datetime64[ns]
Date 列前五行：
 0   2016-01-01
1   2016-04-01
2   2016-07-01
3   2016-10-01
4   2017-01-01
Name: Date, dtype: datetime64[ns]


In [34]:
### Pre-and post-covid 
# 2) Winsorize specified columns at the 1st and 99th percentile
to_winsor = [
    "Current ratio",
    "Interest coverage ratio",
    "ROA",
    "ROE",
    "Leverage ratio",
    "Market to book ratio",
    "Risk free rate",
    "CDS spread"
]

for col in to_winsor:
    q_low  = df[col].quantile(0.01)
    q_high = df[col].quantile(0.99)
    df[col] = df[col].clip(lower=q_low, upper=q_high)

# 3) Log-transform Total assets and Market value
for col in ["Total assets", "Market value"]:
    # if there’s a chance of zero or negative values, add a small constant:
    df["log_" + col.replace(" ", "_").lower()] = np.log(df[col].clip(lower=1e-6))

C:\Users\jiaju\AppData\Local\Temp\ipykernel_12036\4005449826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].clip(lower=q_low, upper=q_high)
C:\Users\jiaju\AppData\Local\Temp\ipykernel_12036\4005449826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].clip(lower=q_low, upper=q_high)
C:\Users\jiaju\AppData\Local\Temp\ipykernel_12036\4005449826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [35]:
df.columns = df.columns.str.replace(' ', '_')

In [36]:

# 1. Convert date to datetime if needed
df['Date'] = pd.to_datetime(df['Date'])

# 2. Set panel index
df = df.set_index(['COMPANY', 'Date'])

# 3. Independent variables (ESG pillars)
ivs = [
    'Environment_Pillar_Score',
    'Social_Pillar_Score',
    'Governance_Pillar_Score',
    'ESG_Controversies_Score'
]

# 4. Control variables
controls = [
    'Current_ratio', 'Interest_coverage_ratio', 'Historical_volatility_5_years',
    'Market_value', 'ROA', 'ROE', 'Leverage_ratio',
    'Market_to_book_ratio', 'log_total_assets', 'log_market_value'
]


C:\Users\jiaju\AppData\Local\Temp\ipykernel_12036\3296754189.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


DateParseError: Unknown datetime string format, unable to parse: Q1 2016, at position 0

In [23]:
def run_fe_models(df_period, period_label):
    summaries = []

    for var in pillar_vars:
        all_vars = [var] + control_vars
        df_model = df_period[[target_var] + all_vars].dropna()

        exog = add_constant(df_model[all_vars])
        endog = df_model[target_var]

        mod = PanelOLS(endog, exog, entity_effects=True)
        res = mod.fit()

        def add_stars(coef, p):
            if p < 0.01:
                return f"{coef:.3f}***"
            elif p < 0.05:
                return f"{coef:.3f}**"
            elif p < 0.1:
                return f"{coef:.3f}*"
            else:
                return f"{coef:.3f}"

        coef = res.params
        pval = res.pvalues
        coef_stars = [add_stars(coef[i], pval[i]) for i in coef.index]

        summary_df = pd.DataFrame({f"{var} ({period_label})": coef_stars}, index=coef.index)

        # Add R² and adjusted R²
        r2 = res.rsquared
        n = res.nobs
        k = len(res.params)
        r2_adj = 1 - ((1 - r2) * (n - 1) / (n - k - 1))

        r2_df = pd.DataFrame({
            f"{var} ({period_label})": [f"{r2:.3f}", f"{r2_adj:.3f}"]
        }, index=["R-squared", "Adj. R-squared"])

        full_summary = pd.concat([summary_df, r2_df])
        summaries.append(full_summary)

    return summaries


In [26]:
# Convert "Q2 2017" into datetime
df['Date'] = pd.PeriodIndex(df['Date'], freq='Q').to_timestamp()

# Define cutoff for 2020 Q2 (stress period starts here)
cutoff = pd.Timestamp("2020-04-01")

# Create period labels
df['period'] = df['Date'].apply(lambda d: 'Calm' if d < cutoff else 'Stress')

# Subset the DataFrame
calm_df = df[df['period'] == 'Calm']
stress_df = df[df['period'] == 'Stress']

# Run FE model summaries
calm_tables = run_fe_models(calm_df, '2016Q1–2019Q4')
stress_tables = run_fe_models(stress_df, '2020Q2–2024Q4')

# Combine all results
all_tables = calm_tables + stress_tables
final_table = pd.concat(all_tables, axis=1)

# Display and save
pd.set_option("display.max_columns", None)
print(final_table)
final_table.to_excel("FE_split_summary_stress_vs_calm.xlsx")


KeyError: 'Date'

In [25]:

calm_df = df[df['Date'] == 'Calm']
stress_df = df[df['Date'] == 'Stress']

calm_tables = run_fe_models(calm_df, '2016Q1–2019Q4')
stress_tables = run_fe_models(stress_df, '2020Q2–2024Q4')

all_tables = calm_tables + stress_tables
final_table = pd.concat(all_tables, axis=1)

# Display and save
pd.set_option("display.max_columns", None)
print(final_table)
final_table.to_excel("FE_split_summary_stress_vs_calm.xlsx")


KeyError: 'Date'